In [30]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import albumentations as albu
import torch
import segmentation_models_pytorch as smp
import shutil
import json

In [31]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cpu'

In [3]:
DATA_DIR = './'
# x_test_dir = os.path.join(DATA_DIR, 'full_training_data/all_valid_imgs')
# y_test_dir = os.path.join(DATA_DIR, 'full_training_data/all_valid_masks')
x_test_dir = os.path.join(DATA_DIR, 'all_eval_imgs')
y_test_dir = os.path.join(DATA_DIR, 'all_eval_masks')

In [4]:
df = pd.read_csv('./tile_meta.csv')
eval_ids = set(df.loc[(df['source_wsi']==3) | (df['source_wsi']==4)]['id'].values)
print(len(eval_ids))
if os.path.exists('./all_eval_imgs'):
  shutil.rmtree('./all_eval_imgs')
if os.path.exists('./all_eval_masks'):
  shutil.rmtree('./all_eval_masks')
os.mkdir('./all_eval_imgs')
os.mkdir('./all_eval_masks')
for f in os.listdir(x_test_dir):
  if f.split('.')[0] in eval_ids:
    shutil.copy(f'{x_test_dir}/{f}', f'./all_eval_imgs/{f}')
    shutil.copy(f'{y_test_dir}/{f}', f'./all_eval_masks/{f}')

681


In [5]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

In [6]:
class HubMapDataset(BaseDataset):
    """Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = ['unlabelled', 'blood_vessel']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return self.images_fps[i], image, mask
        
    def __len__(self):
        return len(self.ids)

In [7]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=512, min_width=352, always_apply=True, border_mode=0),
        albu.RandomCrop(height=512, width=352, always_apply=True),

        albu.GaussNoise(p=0.2),
        albu.Perspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightnessContrast(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.Sharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomBrightnessContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(512, 512)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [8]:
CLASSES = ['blood_vessel']
ENCODER = 'efficientnet-b7'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid'

In [9]:
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [10]:
# Inference
DEVICE = 'cpu'
best_model = torch.load('./models/best_model.pth', map_location=torch.device('cpu'))
best_model = best_model.to(DEVICE)

In [11]:
CLASSES = ['blood_vessel']
test_dataset = HubMapDataset(
    x_test_dir, 
    y_test_dir,
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

test_loader = DataLoader(test_dataset)

In [12]:
# for i in range(3):
#   _, img, mask = test_dataset[i]
#   visualize(image=img.transpose(1,2,0), mask=mask.squeeze())

In [13]:
# target_dataset = test_dataset
# for i in range(3):
#     _, image, gt_mask = target_dataset[i]
#     print(image.shape, gt_mask.shape)
    
#     gt_mask = gt_mask.squeeze()
    
#     x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
#     print(x_tensor.shape)
#     pr_mask = torch.sigmoid(best_model.predict(x_tensor))
#     print(pr_mask.shape)
#     pr_mask = (pr_mask.squeeze().cpu().numpy().round())
#     print(pr_mask.shape)
    
#     visualize(
#         image=image.transpose(2,1,0), 
#         ground_truth_mask=gt_mask, 
#         predicted_mask=pr_mask
#     )

In [14]:
import cv2
import numpy as np
import base64
from pycocotools import _mask as coco_mask
import typing as t
import zlib

def extract_polygon_masks(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    masks = []
    
    for contour in contours:
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        if approx.shape[0] >= 3:
            polygon_mask = np.zeros_like(mask)
            cv2.drawContours(polygon_mask, [approx], 0, 1, -1)
            masks.append(polygon_mask.astype('bool'))
    
    return masks

def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool_:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)
  
  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str

def calculate_iou(target_polygon_masks, output_polygon_mask):
  max_iou_score = 0
  for target_polygon_mask in target_polygon_masks:
    intersection = np.logical_and(target_polygon_mask, output_polygon_mask).sum()
    union = np.logical_or(target_polygon_mask, output_polygon_mask).sum()
    iou_score = float(intersection)/union
    max_iou_score = max(max_iou_score, iou_score)
  return max_iou_score

In [50]:
import time
def generate_map_score(model, device, dataloader, iou_thresh=0.6):
    model.eval()
    num_batches = len(dataloader)
    print(f'Processing a total of {num_batches} images for map score')
    map_score_info = []
    start_time = time.time()
    total_map_score_num = 0
    total_map_score_denom = 0
    # Disable gradient calculation
    with torch.no_grad():
        # Iterate over the validation dataset
        for batch_idx, (img_file, inputs, targets) in enumerate(dataloader):
            cur_dict = dict()
            img_id = img_file[0].split('/')[-1].split('.')[0]
            cur_dict['id'] = img_id
            cur_dict['height'] = 512
            cur_dict['width'] = 512
            iou_values = []
            inputs = inputs.to(device)
            targets = targets.to(device)
            targets = targets.squeeze().numpy().astype('uint8')
            outputs = torch.sigmoid(model(inputs)).squeeze().numpy()
            outputs_thresh = (outputs > 0.5).astype('uint8')
            output_polygon_masks = extract_polygon_masks(outputs_thresh)
            target_polygon_masks = extract_polygon_masks(targets)
            map_score = 0
            for output_polygon_mask in output_polygon_masks:
              output_polygon_mask_conf = round(((output_polygon_mask * outputs).sum())/(output_polygon_mask.sum()), 2)
              iou_val = calculate_iou(target_polygon_masks, output_polygon_mask)
              match_val = 1 if iou_val > iou_thresh else 0
              iou_values.append((match_val, output_polygon_mask_conf, iou_val))
              map_score += match_val * output_polygon_mask_conf
              total_map_score_num += match_val * output_polygon_mask_conf
              total_map_score_denom += output_polygon_mask_conf
            cur_dict['iou_values'] = iou_values
            cur_dict['map_score'] = map_score
            map_score_info.append(cur_dict)
            if batch_idx % 50 == 0:
              print(f'On batch {batch_idx} and finished in {(time.time()-start_time)} seconds')
              start_time = time.time()
    return map_score_info, total_map_score_num/total_map_score_denom

In [51]:
map_score_info, total_map_score = generate_map_score(best_model, DEVICE, test_loader)

Processing a total of 132 images for map score
On batch 0 and finished in 1.6234540939331055 seconds
On batch 50 and finished in 83.79900550842285 seconds
On batch 100 and finished in 98.66336393356323 seconds


In [52]:
total_map_score

0.4354961541390574

In [32]:
# bbox file - ImageID,LabelName,XMin,XMax,YMin,YMax,IsGroupOf
# labels file - ImageID,LabelName,Confidence
# segmentations file - ImageID,LabelName,ImageWidth,ImageHeight,XMin,YMin,XMax,YMax,IsGroupOf,Mask
# predictions file - ImageID,ImageWidth,ImageHeight,LabelName,Score,Mask

In [33]:
import cv2
import numpy as np
import base64
from pycocotools import _mask as coco_mask
import typing as t
import zlib

def extract_polygon_masks(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    masks = []
    
    for contour in contours:
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        if approx.shape[0] >= 3:
            polygon_mask = np.zeros_like(mask)
            cv2.drawContours(polygon_mask, [approx], 0, 1, -1)
            masks.append(polygon_mask.astype('bool'))
    
    return masks

def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool_:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)
  
  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str

In [34]:
with open(f'./polygons.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
tiles_dicts = []
for json_str in json_list:
    tiles_dicts.append(json.loads(json_str))

In [35]:
tgt_ids = [x.split('.')[0] for x in os.listdir('./sample_valid_imgs')]
tgt_ids

['fd569e285d3d', 'fdc723893ce2', 'fdd133458d88']

In [36]:
tiles_dicts_new = [x for x in tiles_dicts if x['id'] in tgt_ids]

In [37]:
len(tiles_dicts_new)

3

In [38]:
bbox_dicts = []
segfile_dicts = []
labels_info = set()
img_width = 512
img_height = 512
for tiles_dict in tiles_dicts_new:
  img_id = tiles_dict['id']
  base_image = cv2.imread(f'./sample_valid_imgs/{img_id}.png')
  for annot in tiles_dict['annotations']:
    if annot['type'] == 'blood_vessel':
      blood_vessel_masked_image = np.zeros((512, 512))
      cur_dict = dict()
      cur_segfile_dict = dict()
      cur_dict['ImageID'] = img_id
      cur_dict['LabelName'] = annot['type']
      coords = annot['coordinates'][0]
      cv2.fillPoly(blood_vessel_masked_image, pts=[np.array(coords)], color=1)
      encoded_mask = encode_binary_mask(blood_vessel_masked_image.astype('bool')).decode('utf-8')
      x_vals = [x[0] for x in coords]
      y_vals = [x[1] for x in coords]
      x_min = float(min(x_vals))/img_width
      x_max = float(max(x_vals))/img_width
      y_min = float(min(y_vals))/img_height
      y_max = float(max(y_vals))/img_height
      cur_dict['XMin'] = x_min
      cur_dict['XMax'] = x_max
      cur_dict['YMin'] = y_min
      cur_dict['YMax'] = y_max
      cur_dict['IsGroupOf'] = 0
      cur_segfile_dict['ImageID'] = img_id
      cur_segfile_dict['LabelName'] = annot['type']
      cur_segfile_dict['ImageWidth'] = img_width
      cur_segfile_dict['ImageHeight'] = img_height
      cur_segfile_dict['XMin'] = x_min
      cur_segfile_dict['XMax'] = x_max
      cur_segfile_dict['YMin'] = y_min
      cur_segfile_dict['YMax'] = y_max
      cur_segfile_dict['IsGroupOf'] = 0
      cur_segfile_dict['Mask'] = encoded_mask
      bbox_dicts.append(cur_dict)
      segfile_dicts.append(cur_segfile_dict)
      cur_labels_info = (img_id, annot['type'], 1)
      if cur_labels_info not in labels_info:
        labels_info.add(cur_labels_info)

In [39]:
bbox_dicts_df = pd.DataFrame.from_dict(bbox_dicts)
bbox_dicts_df.to_csv('./map_input_data/segmentation_bbox.csv', index=False)

In [40]:
labels_info_df = pd.DataFrame(list(labels_info), columns=['ImageID', 'LabelName', 'Confidence'])
labels_info_df.to_csv('./map_input_data/segmentation_labels.csv', index=False)

In [41]:
segfile_df = pd.DataFrame.from_dict(segfile_dicts)
segfile_df.to_csv('./map_input_data/segmentation_masks.csv')

In [42]:
class HubMapDataset(BaseDataset):
    """Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = ['unlabelled', 'blood_vessel']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return self.images_fps[i], image, mask
        
    def __len__(self):
        return len(self.ids)

In [43]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=512, min_width=352, always_apply=True, border_mode=0),
        albu.RandomCrop(height=512, width=352, always_apply=True),

        albu.GaussNoise(p=0.2),
        albu.Perspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightnessContrast(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.Sharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomBrightnessContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(512, 512)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [44]:
DEVICE = 'cpu'
best_model = torch.load('./models/best_model_no_preproc_30_epochs.pth', map_location=torch.device('cpu'))
best_model = best_model.to(DEVICE)

In [45]:
CLASSES = ['blood_vessel']
test_dataset = HubMapDataset(
    './sample_valid_imgs', 
    './sample_valid_masks', 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

test_loader = DataLoader(test_dataset)

In [49]:
import time
model = best_model
dataloader = test_loader
device = 'cpu'
model.eval()
num_batches = len(dataloader)
print(f'Processing a total of {num_batches} images for submission')
prediction_dicts = []
start_time = time.time()
# Disable gradient calculation
with torch.no_grad():
    # Iterate over the validation dataset
    for batch_idx, (img_file, inputs, targets) in enumerate(dataloader):
        cur_dict = dict()
        img_id = img_file[0].split('/')[-1].split('.')[0]
        cur_dict['ImageID'] = img_id
        cur_dict['ImageWidth'] = 512
        cur_dict['ImageHeight'] = 512
        cur_dict['LabelName'] = 'blood_vessel'
        inputs = inputs.to(device)
        outputs = torch.sigmoid(model(inputs)).squeeze().numpy()
        outputs_thresh = (outputs > 0.5).astype('uint8')
        polygon_masks = extract_polygon_masks(outputs_thresh)
        for polygon_mask in polygon_masks:
          cur_dict = dict()
          cur_dict['ImageID'] = img_id
          cur_dict['ImageWidth'] = 512
          cur_dict['ImageHeight'] = 512
          cur_dict['LabelName'] = 'blood_vessel'
          polygon_mask_conf = round(((polygon_mask * outputs).sum())/(polygon_mask.sum()), 2)
          polygon_mask_string = encode_binary_mask(polygon_mask).decode('utf-8')
          cur_dict['Score'] = polygon_mask_conf
          cur_dict['Mask'] = polygon_mask_string
          prediction_dicts.append(cur_dict)
        if batch_idx % 50 == 0:
          print(f'On batch {batch_idx} and finished in {(time.time()-start_time)} seconds')
          start_time = time.time()
    predictions_df = pd.DataFrame.from_dict(prediction_dicts)
    predictions_df.to_csv('./map_input_data/seg_preds.csv', index=False)

Processing a total of 3 images for submission
On batch 0 and finished in 1.4885179996490479 seconds
